In [1]:
import json
with open('GL Bot.json') as file:
    Corpus = json.load(file)

In [2]:
import warnings
warnings.filterwarnings("ignore")

import tflearn
import random
import pickle
import numpy

import tkinter as tk
from tkinter import *
import os
from tkinter import filedialog

FORMAT = "utf-8"
import nltk

text_contents=dict()
nltk.download('punkt',quiet=True)

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


True

In [3]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from nltk.stem import WordNetLemmatizer #word lemmatizer class
lemma = WordNetLemmatizer()
try:
    with open("data.pickle1", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
#  Fetching the words and tags from the corpus
    for intent in Corpus["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
# print only one tag for all the pattern within that tag
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
# Stemming the word and convert in into lower case
    words = [lemma.lemmatize(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)
# Preparing Training and Output data
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [lemma.lemmatize(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle1", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [4]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn1")
except:
    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn1")

Training Step: 15999  | time: 0.052s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9247 -- iter: 120/128
Training Step: 16000  | time: 0.060s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9322 -- iter: 128/128
--
INFO:tensorflow:F:\Great learning\Capstone\model.tflearn1 is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
def greet_res(text):
    text=text.lower()
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)
# bot response
def bot_ress(usr_input):
        usr_input = usr_input.lower()
        bot_res = ''
        results = model.predict([bag_of_words(usr_input, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in Corpus["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        bot_res = random.choice(responses)                
        return bot_res

def widget_get():
    text_widget = root.nametowidget(textcon)
    return text_widget.get('1.0','end-1c')

In [9]:
def saveas(event=None):
    global file_path,filename
    file_path= filedialog.asksaveasfilename( defaultextension=".txt")
    try:
        filename=os.path.basename(file_path)
        root.title(f"Chat Bot - {filename}")
        content=widget_get()
        with open(file_path ,"w") as file:
            file.write(content)
            text_contents[str(textcon)]=hash(content)
            print("Operation successfull")
    except(FileNotFoundError):
        print("Operation not successfull")
        return None
file_path =None

def save(event=None):
    global file_path,filename
    try:
        if(file_path is None):
            file_path = filedialog.asksaveasfilename(defaultextension=".txt")
        filename=os.path.basename(file_path)
        root.title(f"Chat Bot - {filename}")
        content=widget_get()
        with open(file_path ,"w") as file:
            file.write(content)
            text_contents[str(textcon)] = hash(content)
            print("Operation successfull")
    except(FileNotFoundError):
        print("Operation not successfull")
        return None

def new(event=None):
    textcon.delete('2.0', 'end-1c')
    global  file_path,filename
    file_path = None
    content = widget_get()
    text_contents[str(textcon)] = hash(content)
    filename=None

def clear(event=None):
    textcon.delete('2.0', 'end-1c')
    content = widget_get()
    text_contents[str(textcon)] = hash(content)
    
def fopen(event=None):
    global file_path,filename
    file_path = filedialog.askopenfilename(defaultextension=".txt")
    try:
        filename = os.path.basename(file_path)
        root.title(f"Chat Bot - {filename}")
        text_widget = root.nametowidget(textcon)
        with open(file_path, "r") as file:
            content=file.read()
            textcon.delete('1.0', 'end-1c')
            text_contents[str(textcon)] = hash(content)
            text_widget.insert(END,content)
            print("Operation successfull")
    except(FileNotFoundError):
        print("Operation not successfull")
        return None

exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def send(event=None):
    usr_input = message.get()
    usr_input = usr_input.lower()
    textcon.insert(END, f'User: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        textcon.config(fg='Yellow')
        textcon.insert(END,"Bot:Ok bye! Chat with you later\n")
        return root.destroy()
    else:
        textcon.config(fg='yellow')
        if greet_res(usr_input) != None:
            lab=f"Bot: {greet_res(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)
        else:
            lab = f"Bot: {bot_ress(usr_input)}"+'\n'
            textcon.insert(END, lab)
            mes_win.delete(0,END)

In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [lemma.lemmatize(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [11]:
root=tk.Tk()
filename="Untitled.txt"
root.title(f"Chat Bot - Untitled.txt")
root.geometry('500x400')

root.resizable(False, False)
main_menu=Menu(root)
file_menu=Menu(root)
file_menu.add_command(label='Open  <Ctrl+O>',command=fopen)
file_menu.add_command(label='New  <Ctrl+N>',command=new)
file_menu.add_command(label='Save  <Ctrl+S>',command=save)
file_menu.add_command(label='Save as <Ctrl+Shift+S>',command=saveas)
edit_menu=Menu(root)
edit_menu.add_command(label='Clear  <Delete>',command=clear)
main_menu.add_cascade(label="File",menu=file_menu)
main_menu.add_cascade(label="Edit",menu=edit_menu)
main_menu.add_command(label="Quit",command=root.destroy)
root.config(menu=main_menu)

message=tk.StringVar()
chat_win=Frame(root,bd=1,bg='black',width=50,height=8)
chat_win.place(x=6,y=6,height=300,width=480)

textcon=tk.Text(chat_win,bd=1,bg='black',width=50,height=8)
textcon.pack(fill="both",expand=True)

mes_win=Entry(root,width=30,xscrollcommand=True,textvariable=message)
mes_win.place(x=6,y=310,height=60,width=366)
mes_win.focus()

textcon.config(fg='yellow')
textcon.tag_config('usr',foreground='white')
textcon.insert(END,"Bot: This is Jarvis to assist you in Industrial safety!\n\n")
mssg=mes_win.get()

button=Button(root,text='Send',bg='grey',activebackground='orange',command=send,width=12,height=5,font=('Arial'))
button.place(x=376,y=310,height=60,width=110)

scrollbar=tk.Scrollbar(textcon)
scrollbar.pack(fill='y')
scrollbar.place(relheight = 1,relx = 1)
scrollbar.config(command = textcon.yview)

content = widget_get()
text_contents[str(textcon)] = hash(content)
root.bind('<Control-s>',save,file_menu)
root.bind('<Control-Shift-s>',saveas,file_menu)
root.bind('<Return>', send,button)
root.bind('<Control-n>', new,file_menu)
root.bind('<Delete>', clear,edit_menu)
root.bind('<Control-o>', fopen,file_menu)
root.mainloop()